In [ ]:
import pandas as pd


In [ ]:
# 1) Load data
df_stock  = pd.read_csv("yfinance_final.csv", parse_dates=["date"])
df_reddit = pd.read_csv("clean_reddit_data.csv", parse_dates=["date"])
# Ensure date column is datetime
df_stock['date'] = pd.to_datetime(df_stock['date'])
df_reddit['date'] = pd.to_datetime(df_reddit['date'])

# Uppercase and replace dots with dashes (like yfinance style)
df_stock["ticker"] = df_stock["ticker"].str.upper().str.replace('.', '-', regex=False)
df_reddit["ticker"] = df_reddit["ticker"].str.upper().str.replace('.', '-', regex=False)

# 2) Define date range and filter both DataFrames
start, end = "2024-02-02", "2025-04-08"
mask_stock = (df_stock['date'] >= start) & (df_stock['date'] <= end)
mask_reddit = (df_reddit['date'] >= start) & (df_reddit['date'] <= end)


df_stock = df_stock.loc[mask_stock].reset_index(drop=True)
df_reddit = df_reddit.loc[mask_reddit].reset_index(drop=True)
print("shapes: ")
print(df_stock.shape)
print(df_reddit.shape)


shapes: 
(11248, 6)
(4347, 4)


In [ ]:
# 3) Build full date-ticker grid
all_dates = pd.DataFrame({'date': df_stock["date"].drop_duplicates().sort_values()})
tickers = df_reddit["ticker"].unique()
full_index = pd.MultiIndex.from_product(
    [tickers, all_dates["date"]],
    names=["ticker", "date"]
).to_frame(index=False)

# 4) Merge with Reddit data and forward fill
df_reddit_full = full_index.merge(df_reddit, on=["ticker", "date"], how="left")
df_reddit_full = df_reddit_full.sort_values(["ticker", "date"])


In [ ]:
# Forward fill missing sentiment and volume
#df_reddit_full["average_sentiment_score"] = df_reddit_full.groupby("ticker")["average_sentiment_score"].ffill()
#df_reddit_full["comment_volume"] = df_reddit_full.groupby("ticker")["comment_volume"].ffill()

# 5) Create lag features (t-1)
df_reddit_full["comment_sentiment_lag1"] = df_reddit_full.groupby("ticker")["average_sentiment_score"].shift(1)
df_reddit_full["comment_volume_lag1"] = df_reddit_full.groupby("ticker")["comment_volume"].shift(1)

# 6) Keep only lagged features for merge
df_reddit_lag = df_reddit_full[[
    "date", "ticker", "comment_sentiment_lag1", "comment_volume_lag1"
]]

# 7) Merge with stock data
df_merged = df_stock.merge(
    df_reddit_lag,
    on=["date", "ticker"],
    how="left"
)

# 8) Check result
print(df_merged.head(100))
print(f"NaNs after merge:\n{df_merged[['comment_sentiment_lag1', 'comment_volume_lag1']].isna().sum()}")

         date ticker  RealizedVol_3d    Target     Volume       Close  \
0  2024-02-02   AAPL        0.018846  0.016407  102518000  184.740829   
1  2024-02-02    ABT        0.007764  0.016340    7575400  109.364326   
2  2024-02-02   ACHR        0.033679  0.028469    3761800    4.940000   
3  2024-02-02    ACN        0.019876  0.019938    1590500  368.389832   
4  2024-02-02   AMZN        0.026586  0.051287  117154900  171.809998   
..        ...    ...             ...       ...        ...         ...   
95 2024-02-06    MCD        0.028023  0.019214    5927200  276.319733   
96 2024-02-06   META        0.125348  0.130211   21655200  452.586975   
97 2024-02-06    MRK        0.027245  0.003801    6153000  122.451012   
98 2024-02-06   MSFT        0.017698  0.016069   18382600  401.674866   
99 2024-02-06   NFLX        0.006264  0.003578    2840300  555.880005   

    comment_sentiment_lag1  comment_volume_lag1  
0                      NaN                  NaN  
1                      

In [ ]:
#REMOVE MRK ONLY ONE POINT OF DATA
df_merged = df_merged[df_merged["ticker"] != "MRK"]
print("MRK in df_merged:", "mrk" in df_merged["ticker"].unique())

missing = df_merged[df_merged["comment_sentiment_lag1"].isna()]
print(missing["ticker"].value_counts().head(10))  # Most affected tickers
print(missing.groupby("ticker")["date"].min().head(10))  # First missing date per ticker

# See how many and what tickers/dates are affected
missing = df_merged[df_merged["comment_sentiment_lag1"].isna()]
print(missing[["date", "ticker"]].drop_duplicates().head(20))
print(f"Missing rows: {len(missing)} / {len(df_merged)}")
print(df_reddit_lag["date"].nunique())
print(df_stock["date"].nunique())
print("merged: ")
print(df_merged["date"].nunique())

#CHECK MISMATCH IN TICKER FORMAT?
stock_tickers = set(df_stock["ticker"])
reddit_tickers = set(df_reddit["ticker"])
print("Tickers in stock but not in reddit:", stock_tickers - reddit_tickers)
print("Tickers in reddit but not in stock:", reddit_tickers - stock_tickers)



#HOW MANY NaNs?
print(f"NaNs after merge:\n{df_merged[['comment_sentiment_lag1', 'comment_volume_lag1']].isna().sum()}")

#REMOVE FIRST DAY BECAUSE OF T - 1
start, end = "2024-02-03", "2025-04-08"
masker = (df_merged['date'] >= start) & (df_merged['date'] <= end)
df_merged = df_merged.loc[masker].reset_index(drop=True)

#HOW MANY NaNs?
print(f"NaNs after merge:\n{df_merged[['comment_sentiment_lag1', 'comment_volume_lag1']].isna().sum()}")


MRK in df_merged: False
ticker
RTX     294
CRM     291
ISRG    290
CVX     289
NFLX    286
IBM     283
WMT     281
ORCL    279
TMO     277
INTU    274
Name: count, dtype: int64
ticker
AAPL    2024-02-02
ABT     2024-02-02
ACHR    2024-02-02
ACN     2024-02-02
AMZN    2024-02-02
BRK-B   2024-02-02
COST    2024-02-02
CRM     2024-02-02
CRWD    2024-02-02
CVX     2024-02-02
Name: date, dtype: datetime64[ns]
         date ticker
0  2024-02-02   AAPL
1  2024-02-02    ABT
2  2024-02-02   ACHR
3  2024-02-02    ACN
4  2024-02-02   AMZN
5  2024-02-02  BRK-B
6  2024-02-02   COST
7  2024-02-02    CRM
8  2024-02-02   CRWD
9  2024-02-02    CVX
10 2024-02-02   GOOG
11 2024-02-02  GOOGL
12 2024-02-02     HD
13 2024-02-02    IBM
14 2024-02-02   INTU
15 2024-02-02   ISRG
16 2024-02-02    JNJ
17 2024-02-02    JPM
18 2024-02-02     KO
19 2024-02-02    MCD
Missing rows: 7488 / 11419
296
296
merged: 
296
Tickers in stock but not in reddit: set()
Tickers in reddit but not in stock: set()
NaNs after merge:
c

In [ ]:
missing = df_merged[df_merged["comment_sentiment_lag1"].isna()]
total_counts = df_merged["ticker"].value_counts()
missing_counts = missing["ticker"].value_counts()
coverage = (
    pd.DataFrame({
        "total": total_counts,
        "missing": missing_counts
    })
    .fillna(0)  # if some tickers have no missing values
)
coverage["missing_pct"] = coverage["missing"] / coverage["total"]
good_tickers = coverage[coverage["missing_pct"] <= 0.7].index
df_filtered = df_merged[df_merged["ticker"].isin(good_tickers)].copy()

print(f"Kept {len(good_tickers)} tickers out of {len(coverage)}")
print(f"Filtered dataset shape: {df_filtered.shape}")



Kept 15 tickers out of 37
Filtered dataset shape: (4892, 8)


In [ ]:
print(df_filtered.head())

print(df_filtered['ticker'].unique())


# Add missing indicators
df_filtered["reddit_sentiment_missing"] = df_filtered["comment_sentiment_lag1"].isna().astype(int)
# Fill NaNs with zeros
df_filtered["reddit_sentiment_lag1"] = df_filtered["comment_sentiment_lag1"].fillna(0.0)
df_filtered["reddit_volume_lag1"] = df_filtered["comment_volume_lag1"].fillna(0.0)



         date ticker  RealizedVol_3d    Target    Volume       Close  \
0  2024-02-05   AAPL        0.016407  0.009968  69668800  186.559891   
4  2024-02-05   AMZN        0.051287  0.043985  55081300  170.309998   
5  2024-02-05  BRK-B        0.010539  0.005626   3651900  390.760010   
6  2024-02-05   COST        0.012018  0.006546   2276900  707.288696   
10 2024-02-05   GOOG        0.045971  0.002081  29254400  144.246780   

    comment_sentiment_lag1  comment_volume_lag1  
0                 0.252806                 33.0  
4                 0.318450                 12.0  
5                      NaN                  NaN  
6                 0.288800                  2.0  
10                0.663400                  2.0  
['AAPL' 'AMZN' 'BRK-B' 'COST' 'GOOG' 'GOOGL' 'HD' 'KO' 'META' 'MSFT'
 'NVDA' 'PG' 'PLTR' 'PM' 'TSLA']


In [ ]:
print(df_filtered.head())
print(df_filtered.shape)
df_filtered[['date','ticker','RealizedVol_3d','Target','reddit_sentiment_lag1','reddit_volume_lag1','reddit_sentiment_missing']].to_csv('yahooredditcombined.csv', index = False)

         date ticker  RealizedVol_3d    Target    Volume       Close  \
0  2024-02-05   AAPL        0.016407  0.009968  69668800  186.559891   
4  2024-02-05   AMZN        0.051287  0.043985  55081300  170.309998   
5  2024-02-05  BRK-B        0.010539  0.005626   3651900  390.760010   
6  2024-02-05   COST        0.012018  0.006546   2276900  707.288696   
10 2024-02-05   GOOG        0.045971  0.002081  29254400  144.246780   

    comment_sentiment_lag1  comment_volume_lag1  reddit_sentiment_missing  \
0                 0.252806                 33.0                         0   
4                 0.318450                 12.0                         0   
5                      NaN                  NaN                         1   
6                 0.288800                  2.0                         0   
10                0.663400                  2.0                         0   

    reddit_sentiment_lag1  reddit_volume_lag1  
0                0.252806                33.0  
4       